In [3]:
# remove name entity
from nltk.corpus import names
all_names = set(names.words())
# lemmaization

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def letters_only(word):
  return word.isalpha()

# put all together to clean texts
def clean_text(doc):
    cleaned_doc = []
    for word in doc.split(' '): # split doc. by blank (' ')
        word = word.lower() # ABD -> abd
  
        if letters_only(word) and word not in all_names and len(word) > 2: # remove number and punc. and name entity
            cleaned_doc.append(lemmatizer.lemmatize(word, "v"))
    return ' '.join(cleaned_doc) 

In [4]:
import pandas as pd
train_df = pd.read_csv("TRAIN.csv")
test_df = pd.read_csv("TEST.csv")
x_train = train_df.iloc[:, 2]
y_train = train_df.iloc[:,-1].astype(int)
x_train = [clean_text(doc) for doc in x_train]
x_test = test_df["Post"]
x_test = [clean_text(doc) for doc in x_test]
print(len(x_train), len(x_test))

350 150


In [5]:
import math
import numpy as np
from nltk.corpus import stopwords 

class Vectorizer:
    def __init__(self, max_len = 100000, tfidf = False, stop_word = False, max_df = float("inf"),
                 min_df = -float("inf"), bigram = False):
        self.max_len = max_len
        self.tfidf = tfidf
        self.stop_word = stop_word
        self.max_df = max_df
        self.ngram = bigram
        self.min_df = min_df
    
    def vocab_(self):
        return self.word_count
    
    def n_gram(self, text, n):
        fi = text.split(" ")
        ret_list = [None for i in range(len(fi) -n +1)]
        for i in range(len(fi) -n +1):
            add = ""
            for k in range(n-1):
                add += fi[i + k]
                add += " "
            add += fi[i+n-1]
            ret_list[i] = add
        return ret_list
    
    
    def make_vector(self, word_count):
        self.word_count = word_count
        sort_by_count = sorted(word_count.items(), key = lambda x:x[1][0], reverse = True)
        how_many = len(sort_by_count)
        if self.max_len > how_many:
            how_many = how_many
        elif self.max_len <= how_many:
            how_many = self.max_len
            
        count = 0
        index_of_words = {}
        self.idfs = []
        for key, value in sort_by_count:
            if count >= how_many:
                break
            if self.stop_word == True:
                if key not in stopwords.words("english"):
                    index_of_words[key] = count
                    if self.tfidf == True:
                        self.idfs.append(value[2])
                    count += 1
                else:
                    continue
            else:
                index_of_words[key] = count
                if self.tfidf == True:
                    self.idfs.append(value[2])
                count += 1

        self.max_len = how_many
        self.index_of_words = index_of_words
                
        
    def tfidfVect(self, word_count):
        nD = len(self.texts)
        for key, value in sorted(word_count.items(), key = lambda x:x[1][0], reverse = True):
            idf = math.log(nD/(1+value[1]))
            word_count[key].append(idf)
            word_count[key][0] = value[0] * idf
        
        self.make_vector(word_count)
        
    def countVect(self, word_count):
        self.make_vector(word_count)
        
    def fit(self, texts):
        self.texts = texts
        word_count = {}
            
        #word_count = {단어: [전체단어개수,단어가 나온 문장개수]}
        for text in texts:
            word_in_text = []
            ts = text.split()
            if self.ngram != False:
                ts += self.n_gram(text, 2)
            for word in ts:
                if word not in word_count.keys():
                    word_count[word] = [1, 1]
                    word_in_text.append(word)
                else:
                    word_count[word][0] += 1
                    word_in_text.append(word)
            
            for i in set(word_in_text):
                word_count[i][1] += 1
        
        chaier = []
        for key, value in word_count.items():
            if value[0] > self.max_df and value[0] < self.min_df:
                chaier.append(key)
        for i in chaier:
            del word_count[i]
        
        if self.tfidf:
            self.tfidfVect(word_count)
        
        else:
            self.countVect(word_count)
        
    def transform(self, texts):
        ret_list = [[0 for i in range(self.max_len)] for i in range(len(texts))]
        for i in range(len(texts)):
            text = texts[i]
            ts = text.split()
            if self.ngram != False:
                text = self.n_gram(text, 2)
                ts = text
            for word in ts:
#                 print(word)
                if word in self.index_of_words.keys():
#                     print(self.index_of_words[word])
                    ret_list[i][self.index_of_words[word]] += 1
        
        ret_list = np.array(ret_list)
        idfs = np.array(self.idfs)
        print(ret_list.shape, idfs.shape)
        if self.tfidf == True:
            ret_list = ret_list * idfs
        
        return ret_list

In [53]:
class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None


    def fit_(self, X, y_):
        n_samples, n_features = X.shape
        
#         y_ = np.where(np.array(y) == 0, -1, 1)
        # Problem 1: label을 1 또는 -1로 변환해주세요 (decision boundary)
        
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx]*(np.dot(self.w, np.array(x_i)) - self.b) >=1
                # Problem 2: 위의 수식을 참고하여 코드를 완성해주세요
                
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)

                else:
                    pass
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(y_[idx], np.array(x_i)))
                     # Problem 3a: 위의 수식을 참고하여 코드를 완성해주세요
                    self.b -= self.lr * (y_[idx])
                    # Problem 3b: 위의 수식을 참고하여 코드를 완성해주세요
    
    
    def fit(self, x_tempo, y_tempo):
        self.sortByindex =[None for i in range(len(set(y_tempo)))]
        self.x_tempo = x_tempo
        self.y_tempo = y_tempo
        print(x_tempo.shape[0], y_tempo.shape[0])
        for value in range(len(set(y_tempo))):
            y_temp = []
            for i in range(len(y_tempo)):
                if y_tempo[i] == value:
                    y_temp.append(1)
                else:
                    y_temp.append(-1)
                    
#             print(y_temp)
                    
            self.sortByindex[value] = y_temp
                
    def predict__(self, X):
        approx = np.dot(X, self.w) + self.b
        return approx
#         return approx  
        
    def predict_(self, X):
        approx = np.dot(X, self.w) - self.b
        return approx
#         return approx

    def predict(self, X):
#         self.fin = [[-1 for i in range(len(set(self.y_tempo)))] for i in range(X.shape[0])]
        self.fin = [0 for i in range(X.shape[0])]
#         print(self.fin)
#         print(X.shape)
        real_fin = []
        for value in range(len(set(self.y_tempo))):
            x_temp = self.x_tempo
            y_temp = np.array(self.sortByindex[value])
#             print(x_temp, y_temp)
            self.fit_(x_temp, y_temp)
            prediction = clf.predict__(X)
            real_fin.append(list(prediction))
#             print(prediction)
#             for i in range(prediction.shape[0]):
#                 if int(prediction[i]) == 1:
#                     self.fin[i]= value
#                 self.fin[i][value] = prediction
        real = []
        
        for text in range(X.shape[0]):
            maxer = -float("inf")
            maxer_idx = -1
            for label in range(len(real_fin)):
#                 print(label)
                if maxer < real_fin[label][text]:
                    maxer_idx = label
                    maxer = real_fin[label][text]
            real.append(maxer_idx)
        return np.array(real)

In [54]:
def train_test_spliter(X, y, test_rate = 0.858):
    standard = int(len(X)* test_rate)
    return (X[:standard], X[standard:]), (y[:standard], y[standard:])

In [55]:
(train_x_train,test_x_train), (train_y_train, test_y_train) = train_test_spliter(x_train, y_train)

In [65]:
cv = Vectorizer(max_len = 1500, tfidf = True, stop_word = True, max_df = 500,
                 min_df = 10, bigram = True)
cv.fit(x_train)
# print(cv.index_of_words)
# print(x_train)
term_docs_train = cv.transform(train_x_train)
term_docs_test = cv.transform(test_x_train)

(300, 1000) (1000,)
(50, 1000) (1000,)


In [72]:
clf = SVM(learning_rate=0.01, lambda_param=0.001, n_iters=1000)
clf.fit(term_docs_train, train_y_train)
y_pred = clf.predict(term_docs_test)

300 300


In [73]:
def accuracy_function(a, b):
    b = list(b)
    a = list(a)
    count = 0
    for i in range(len(b)):
        if a[i] == b[i]:
            count+= 1
    return count / len(b)

In [74]:
accuracy_function(y_pred, test_y_train)

0.24

In [60]:
cv = Vectorizer(1150, tfidf = True, stop_word = False, max_df=500, min_df = 5, bigram = True)
cv.fit(x_train)
term_docs_train = cv.transform(x_train)
test_term_docs = cv.transform(x_test)

(350, 1150) (1150,)
(150, 1150) (1150,)


In [61]:
test_term_docs.shape

(150, 1150)

In [62]:
clf = SVM(learning_rate=0.001, lambda_param=0.001, n_iters=1000)
clf.fit(term_docs_train, y_train)
test_pred = clf.predict(test_term_docs)

350 350


In [63]:
test_pred.shape[0]

150

In [64]:
pd.DataFrame(test_pred).to_csv("2018310412.csv", index=False)